In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Matrix Factorization
I will give two vertions: Rank and Scores.

## Rank

In [2]:
df_scores = pd.read_csv("/home/zhangqiyuan/bench_test/all_benchmark_score.csv", index_col=0)
df_scores.replace("-", np.nan, inplace=True)
#df_ranks = df_scores.rank(axis=1, method='min', ascending=False, na_option='keep')

score_data = np.genfromtxt("/home/zhangqiyuan/bench_test/all_benchmark_score.csv", delimiter=',', skip_header=1)
non_nan_positions = [(idt, col) for idt, (idx, row) in enumerate(df_scores.iterrows()) for col in df_scores.columns if pd.notna(row[col])]
#train_set, validate_set = train_test_split(filtered_df, test_size=0.2, random_state=42)
positions_df = pd.DataFrame(non_nan_positions, columns=['idx', 'idy'])
train_positions, validate_positions = train_test_split(positions_df, test_size=0.1, random_state=42)
column_name_to_index = {name: index for index, name in enumerate(df_scores.columns)}
train_array_positions = np.array([(row['idx'], column_name_to_index[row['idy']]) for idx, row in train_positions.iterrows()])
validate_array_positions = np.array([(row['idx'], column_name_to_index[row['idy']]) for idx, row in validate_positions.iterrows()])
score_data = score_data[:, 1:]
score_data

array([[0.238, 0.243, 0.264, ..., 0.525, 0.53 , 0.386],
       [0.464, 0.581, 0.457, ...,   nan,   nan,   nan],
       [0.238, 0.326, 0.294, ..., 0.749, 0.702, 0.629],
       ...,
       [  nan,   nan,   nan, ..., 0.578, 0.519, 0.397],
       [  nan,   nan,   nan, ..., 0.445, 0.497, 0.312],
       [  nan,   nan,   nan, ..., 0.088, 0.117, 0.023]])

In [4]:


def matrix_factorization(R, P, Q, K, train, steps=5000, alpha=0.0002, beta=0.0000002):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T
    previous_step_e = 100000000
    for step in range(steps):
        print("step:", step)
        for i in range(len(R)):
            for j in range(len(R[i])):
                if not np.isnan(R[i][j]) and np.any(np.isin(train, np.array([i,j]))):
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if not np.isnan(R[i][j]) and np.any(np.isin(train, np.array([i,j]))):

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

                    #for k in range(K):

                    #    e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        print("error:", e)
        
        if e < 0.001 or e > previous_step_e:
            break
        previous_step_e = e
    return P, Q.T

In [5]:
R = score_data
# N: num of User
N = len(R)
print(N)
# M: num of Movie
M = len(R[0])
print(M)
# Num of Features
K = 10

 
P = np.random.rand(N,K)
Q = np.random.rand(M,K)

 

nP, nQ = matrix_factorization(R, P, Q, K, train_positions)

nR = np.dot(nP, nQ.T)

56
88
step: 0
error: 12706.969805141247
step: 1
error: 10703.894148939078
step: 2
error: 9134.449366088686
step: 3
error: 7882.158985416963
step: 4
error: 6867.214583836828
step: 5
error: 6033.434751514793
step: 6
error: 5340.335046972414
step: 7
error: 4758.140260933395
step: 8
error: 4264.553671987842
step: 9
error: 3842.6095682572154
step: 10
error: 3479.2127358518387
step: 11
error: 3164.1246352078083
step: 12
error: 2889.2466023149664
step: 13
error: 2648.1045749314817
step: 14
error: 2435.473063939575
step: 15
error: 2247.0969448927344
step: 16
error: 2079.4830166743754
step: 17
error: 1929.7420148451108
step: 18
error: 1795.4675823795574
step: 19
error: 1674.6426325368245
step: 20
error: 1565.5662374444798
step: 21
error: 1466.796054208393
step: 22
error: 1377.1026244153877
step: 23
error: 1295.4328275203109
step: 24
error: 1220.8804501055806
step: 25
error: 1152.6623298267891
step: 26
error: 1090.0988986199723
step: 27
error: 1032.598221507968
step: 28
error: 979.6428310191054


In [6]:
def rmse(R, P, Q, validate):
    '''
    Calculates the root mean square error (RMSE) between the actual ratings and the predicted ratings.
    
    Parameters:
    - R: numpy array, the actual ratings matrix.
    - P: numpy array, the user feature matrix.
    - Q: numpy array, the item feature matrix.
    
    Returns:
    - float, the RMSE between the actual and predicted ratings.
    '''
    Q = Q.T
    predicted_R = np.dot(P, Q)
    error = 0
    count = 0
    for i in range(len(R)):
        for j in range(len(R[i])):
            if not np.isnan(R[i][j]) and np.any(np.isin(validate, np.array([i,j]))):
                error += pow(R[i][j] - predicted_R[i][j], 2)
                count += 1
    return np.sqrt(error / count)

In [7]:
rmse(R, nP, nQ, validate_array_positions)
with open('score_nP.npy', 'wb') as f:
    np.save(f, nP)
with open('score_nQ.npy', 'wb') as f:
    np.save(f, nQ)

In [ ]:
# visualize prediction score
predicted_R=np.dot(nP, nQ.T)
for position in validate_array_positions:
    print(predicted_R[position[0], position[1]], R[position[0], position[1]])

0.12440092591565069 0.04087906282998202
0.0743179397208763 0.12056795581660754
0.05040932092691371 0.041545480537934616
0.13835535667000184 0.1213977395138023
0.1458919209589081 0.1739523740198809
0.11896049312423186 0.1287797268250796
-0.08999033686310838 -0.03751057082015702
0.1697854248144422 0.18117219884780283
0.10997112028419956 0.12839519600720245
0.14809526035976858 0.16104947925902854
0.13240814815792154 0.10879970252293641
0.11021515347487469 0.13971546498847026
0.07760518931986408 0.12294200839246197
0.12152717191444516 0.11232887703042732
0.09856783892628042 0.07342710143547414
0.07032489342787018 0.11378494845702344
0.09052807026524581 0.04477797729367133
0.08648748046870795 0.12128491428786353
0.05225459474313392 0.00553862933853129
0.12880369932803112 0.17074191120975285
0.13377306199179537 0.1847448637460305
0.08862664970157194 0.11447442300220795
0.15732212648779215 0.17640568586860553
0.09621289066417563 0.13555372773349456
0.058293700905506285 0.029028622963186843
0.

In [ ]:
# score to rank
nan_mask = np.isnan(score_data)
predict_score = np.dot(nP, nQ.T)
predict_score[nan_mask] = np.nan
predict_score = pd.DataFrame(predict_score)
predict_rank = predict_score.rank(axis=1, method='min', ascending=False, na_option='keep')
predict_rank

,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,86,87
0,74.0,81.0,79.0,80.0,75.0,71.0,60.0,78.0,62.0,64.0,...,9.0,21.0,16.0,15.0,1.0,3.0,4.0,27.0,11.0,29.0
1,54.0,62.0,61.0,60.0,59.0,57.0,39.0,63.0,47.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,67.0,80.0,79.0,78.0,75.0,73.0,62.0,77.0,59.0,61.0,...,4.0,17.0,18.0,15.0,6.0,1.0,3.0,22.0,5.0,30.0
3,74.0,81.0,79.0,80.0,72.0,71.0,64.0,78.0,59.0,60.0,...,11.0,18.0,16.0,19.0,5.0,1.0,3.0,25.0,13.0,36.0
4,71.0,78.0,79.0,77.0,75.0,73.0,56.0,80.0,58.0,60.0,...,3.0,19.0,18.0,15.0,6.0,1.0,4.0,24.0,5.0,29.0
5,53.0,62.0,60.0,61.0,57.0,55.0,45.0,59.0,43.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,22.0,32.0,30.0,31.0,18.0,25.0,21.0,28.0,12.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,41.0,48.0,47.0,50.0,45.0,43.0,29.0,49.0,31.0,35.0,...,3.0,16.0,14.0,15.0,8.0,2.0,6.0,17.0,12.0,18.0
8,60.0,63.0,65.0,64.0,57.0,51.0,45.0,61.0,43.0,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,28.0,30.0,NaN,29.0,26.0,24.0,13.0,27.0,23.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rank_data = np.genfromtxt("/home/zhangqiyuan/bench_test/all_bencmark_rank.csv", delimiter=',', skip_header=1)
rank_data = rank_data[:, 1:]
#visualize prediction rank
for position in validate_positions:
    print(predict_rank.iloc[position[0], position[1]], rank_data[position[0], position[1]])

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [ ]:
#evaluate
rank_data = np.genfromtxt("/home/zhangqiyuan/bench_test/all_bencmark_rank.csv", delimiter=',', skip_header=1)
rank_data = rank_data[:, 1:]
error = 0
count = 0
error_1 = []
right = 0
for position in validate_array_positions:
    error += pow(rank_data[position[0], position[1]] - predict_rank.iloc[position[0], position[1]], 2)
    error_1.append(abs(rank_data[position[0], position[1]] - predict_rank.iloc[position[0], position[1]]))
    if abs(rank_data[position[0], position[1]] - predict_rank.iloc[position[0], position[1]]) == 0:
        right+=1
    count += 1
print("Root Mean Squared Error", np.sqrt(error / count), "Mean Absolute Error :", sum(error_1)/count)
print("Prec:", right/count )

[27 50] 8.0 29.0
[ 2 45] 56.0 28.0
[3 3] 80.0 79.0
[38  6] 19.0 28.0
[48 13] 18.0 14.0
[33  0] 35.0 20.0
[43  4] 3.0 40.0
[ 6 15] 15.0 17.0
[13 25] 29.0 27.0
[37  0] 22.0 25.0
[29 41] 13.0 55.0
[29 35] 22.0 5.0
[ 2 19] 47.0 21.0
[ 0 44] 20.0 33.0
[ 0 56] 30.0 58.0
[ 2 24] 65.0 42.0
[26 57] 29.0 50.0
[29 55] 58.0 35.0
[25 17] 67.0 68.0
[20 40] 14.0 4.0
[36 52] 21.0 11.0
[5 5] 55.0 42.0
[36 15] 11.0 18.0
[29 25] 38.0 13.0
[32 66] 66.0 65.0
[ 7 25] 23.0 24.0
[21 12] 57.0 65.0
[21 18] 36.0 60.0
[46 16] 37.0 40.0
[20 23] 23.0 11.0
[ 7 24] 38.0 21.0
[24 20] 39.0 52.0
[13 24] 45.0 21.0
[30 18] 37.0 22.0
[26 44] 15.0 27.0
[25 65] 84.0 66.0
[31  2] 61.0 65.0
[ 2 22] 53.0 12.0
[40 26] 7.0 7.0
[45  5] 26.0 26.0
[23  0] 60.0 69.0
[ 2 37] 24.0 6.0
[ 2 60] 48.0 66.0
[17 27] 2.0 10.0
[31 52] 45.0 58.0
[26 27] 25.0 24.0
[31  1] 63.0 64.0
[34  6] 20.0 26.0
[33 20] 21.0 14.0
[ 4 70] 9.0 2.0
[ 5 46] 12.0 18.0
[13 42] 4.0 12.0
[14 56] 16.0 32.0
[39  0] 24.0 27.0
[30 35] 31.0 13.0
[7 4] 45.0 45.0
[10 31] 1